# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Загрузка-модулей-и-библиотек" data-toc-modified-id="Загрузка-модулей-и-библиотек-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Загрузка модулей и библиотек</a></span></li><li><span><a href="#Функции-используемые-в-проекте" data-toc-modified-id="Функции-используемые-в-проекте-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Функции используемые в проекте</a></span></li><li><span><a href="#Загрузка-данных-и-знакомство-с-ними" data-toc-modified-id="Загрузка-данных-и-знакомство-с-ними-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Загрузка данных и знакомство с ними</a></span></li></ul></li><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Категоризация данных</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></li></ul></div>

### Загрузка модулей и библиотек

In [21]:
!pip install pymystem3

  Using cached pymystem3-0.2.0-py3-none-any.whl (10 kB)


In [31]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
m = Mystem()
import collections
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()


### Функции используемые в проекте

In [28]:
def lem(row):
    l = m.lemmatize(row)
    if 'недвижимость' in l or 'жилье' in l:
        return 'жилье'
    if 'автомобиль' in l:
        return 'авто'
    if 'свадьба' in l:
        return 'свадьба'
    if 'образование' in l:
        return 'учеба'

In [35]:
def children(row):
    if row == 0:
        return 'бездетный'
    if row == 1:
        return 'один ребенок'
    if row == 2:
        return 'два нахлебника'
    else:
        return 'многодетный'


In [36]:
def income(row):
    x = df['total_income'].quantile(0.25)
    y = df['total_income'].quantile(0.75)
        
    if row <= x:
        return 'низкий'
    if row > x and row <= y:
        return 'средний'
    else:
        return 'высокий'

### Загрузка данных и знакомство с ними 

In [1]:
df = pd.read_csv('C:/Users/chuper//datasets/data.csv') #прочитаем файл и сохраним его в df 
df.info() #посмотрим общую информацию о таблице 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
df.head(5) #познакомимся с данными 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [4]:
for row in df: 
      print(df[row].value_counts()) # смотрим какие уникальные значения 

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
среднее   

## Описание данных

0. children — количество детей в семье, тип целое число, есть 47 отрицательных значений и 76 значений у кого 20 детей, столбец  **нужен для вывода по проекту** 
1. days_employed — общий трудовой стаж в днях, дробное число, *содержит пропуски* , *отрицательные значения*, не нужен для выводов по проекту 
2. dob_years — возраст клиента в годах, целое число, есть 101 нулевое значение, не нужен для выводов по проекту 
3. education — уровень образования клиента, категория , проблемы с регистром , не нужен для выводов по проекту  
4. education_id — идентификатор уровня образования, целое число, не нужен для выводов по проекту 
5. family_status — семейное положение, категория ,  **нужен для выводов по проекту**
6. family_status_id — идентификатор семейного положения, целое число , кол-во совпадает с категорией family_status , **нужен для выводов по проекту**
7. gender — пол клиента, категория , есть значение XNA 1, не нужен для выводов по проекту 
8. income_type — тип занятости, категория, **нужен для выводов по проекту**
9. debt — имел ли задолженность по возврату кредитов, целое число, **нужен для выводов по проекту**
10. total_income — ежемесячный доход, дробное число, *содержит пропуски* ,**нужен для выводов по проекту**
11. purpose — цель получения кредита, категория , есть проблемы нужно лематизировать, **нужен для выводов по проекту**

### Вывод

Каждая строка содержит информацию о заемщике, есть пропущенные значения в доходе. Нужно заменить отрицательные значения в кол-ве детей и 20 детей скорее всего опечатка. Список целей для получения кредита нужно лематизировать и переработать. 

Пропущенные значения находятся в столбцах:  
- total_income
Нужно посчитать количество пропусков, отношение к общему числу данных. 

In [5]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
total_income_null = df['total_income'].isnull().sum()
print(total_income_null)

2174


In [7]:
total_income_total = df.loc[:,'total_income'].count()
print(total_income_total)

19351


In [8]:
print('Процент пропущеных значений:{:.1%}'.format(total_income_null/total_income_total)) 

Процент пропущеных значений:11.2%


Высокий процент пропущеных значений в доходах. Пропуски совпадают с пропусками в стаже. Возможно данные повреждены при выгрузке. Поскольку доход количественная величина можно заполнить пропуски медианным значением. Однако для заполнения лучше взять медианное значение для каждого типа занятости (income_type).

## Предобработка данных

### Обработка пропусков

In [9]:
print(df['total_income'].mean())
data_income = df.groupby('income_type')['total_income'].median() #получаем медиану по каждому типу занятости
print(data_income)

167422.3022081719
income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64


In [10]:
df['total_income'] = df.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))

In [11]:
df['total_income'].isnull().sum()

0

Поправим данные в кол-ве детей.Нужно заменить отрицательные значения в кол-ве детей и 20 детей скорее всего ошибка при заполнении

In [12]:
df['children'].value_counts()
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)

In [13]:
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Заполнили пропуски в доходе и убрали опечатки в детях

### Замена типа данных

Данные о доходе представлены в формате float заменим на int64, чтобы доход был целым числом используем метод astype('int') так как метод int() не подходит к обьекту типа series

In [14]:
df['total_income'] = df['total_income'].astype('int')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int32  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


Теперь доход целые числа

### Обработка дубликатов

Найдем есть ли дубликаты методом 

In [16]:
df.duplicated().sum()

54

In [17]:
df = df.drop_duplicates().reset_index(drop = True)

In [18]:
df.duplicated().sum()

0

Больше дубликатов нет, однако стоит заметить, данные этого датасета не содержат ID клиента, и поэтому возможно что у 54 человек совпал доход, семейное положение и прочие параметры что для больших выборок, вполне возможная вещь.

### Лемматизация

In [26]:
purpose_dict = df['purpose'].unique()

In [27]:
lemmas_count = collections.Counter()

for purpose in purpose_dict:
    lemmatize_purpose = m.lemmatize(purpose)
    print(lemmatize_purpose)
    for word in lemmatize_purpose:
        lemmas_count[word] += 1

print()
print(lemmas_count)

['покупка', ' ', 'жилье', '\n']
['приобретение', ' ', 'автомобиль', '\n']
['дополнительный', ' ', 'образование', '\n']
['сыграть', ' ', 'свадьба', '\n']
['операция', ' ', 'с', ' ', 'жилье', '\n']
['образование', '\n']
['на', ' ', 'проведение', ' ', 'свадьба', '\n']
['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n']
['покупка', ' ', 'недвижимость', '\n']
['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n']
['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n']
['недвижимость', '\n']
['строительство', ' ', 'недвижимость', '\n']
['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n']
['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n']
['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n']
['жилье', '\n']
['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n']
['автомобиль', '\n']
['заниматься', ' ', 'образование'

In [32]:
df['purpose_type'] = df['purpose'].progress_apply(lem)

  0%|          | 0/21471 [00:00<?, ?it/s]

In [33]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,авто
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,учеба
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,учеба
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье


### Категоризация данных

Категоризация: 1.проведем категаризацию по кол-ву детей , напишем  функцию которая будет проверять в столбце ['children'] колв-о детей и возвращать категорию : бездетные , 1 ребенок , 2 ребенка , многодетные применим ее . И сгрупируем столбцы ['children'] и ['debt'] в сборной таблице

In [37]:
df['children_category'] = df['children'].apply(children)
children_status = df.pivot_table(index='children_category', columns='debt', values='children', aggfunc='count')

Посчитаем соотношение вернувших кредит и просрочивших в каждой категории. Результаты представим в таблице

In [38]:
children_status['ratio %'] = (children_status[1] / children_status[0]) * 100
children_status['ratio %'] = children_status['ratio %'].astype('int')
children_status.head()

debt,0,1,ratio %
children_category,,,
бездетный,13044,1063,8
два нахлебника,1926,202,10
многодетный,349,31,8
один ребенок,4411,445,10


По семейному положению 

In [39]:
family_status = df.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')
family_status['ratio %'] = (family_status[1] / family_status[0]) * 100
family_status['ratio %'] = family_status['ratio %'].astype('int')

In [40]:
family_status.head(10)

debt,0,1,ratio %
family_status,,,
Не женат / не замужем,2536,274,10
в разводе,1110,85,7
вдовец / вдова,896,63,7
гражданский брак,3775,388,10
женат / замужем,11413,931,8


По уровню дохода. 

Здесь нужно задать категории, учитывая что средняя величина дохода 165295 рублей и учитыавя медианных доход найденый ранее : 
- безработный        131339
- в декрете           53829
- госслужащий        150447
- компаньон          172357
- пенсионер          118514
- предприниматель    499163
- сотрудник          142594
- студент             98201

предлагаю установить 3 категории: 
- низкий доход, меньше чем у 25% заемщиков
- средний доход, как у большенства заемщиков (25-75% заемщиков)
- и высокий свыше 75% заемщиков<br>

Функция для создания этой категории использует квантили распределения. Применим ее. 


In [41]:
df['total_income_cat'] = df['total_income'].apply(income)

In [42]:
income_cat = df.pivot_table(index='total_income_cat', columns='debt', values='total_income', aggfunc='count')
income_cat['ratio %'] = (income_cat[1] / income_cat[0]) * 100
income_cat['ratio %'] = income_cat['ratio %'].astype('int')
income_cat.head(10)

debt,0,1,ratio %
total_income_cat,,,
высокий,4564,345,7
низкий,4941,427,8
средний,10225,969,9


По цели кредита

In [43]:
purpose_type = df.pivot_table(index='purpose_type', columns='debt', values='purpose', aggfunc='count')
purpose_type['ratio %'] = (purpose_type[1] / purpose_type[0]) * 100
purpose_type['ratio %'] = purpose_type['ratio %'].astype('int')
purpose_type.head(10)

debt,0,1,ratio %
purpose_type,,,
авто,3905,403,10
жилье,10032,782,7
свадьба,2149,186,8
учеба,3644,370,10


### Общий вывод 

Есть зависимости возврата кредита в различных категориях данных около 2-3% <br>
- Есть ли зависимость между наличием детей и возвратом кредита в срок?<br>
Да бездетные и многодетные отдают кредит на 2% чаще<br>
- Есть ли зависимость между семейным положением и возвратом кредита в срок?<br>
Люди в разводе и вдовцы платят кредит на 3 % чаще чем неженатые<br>
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?<br>
Люди с высоким доходом возвращают чаще на 2 %<br>
- Как разные цели кредита влияют на его возврат в срок?<br>
Цели на жилье и свадьбу возвращают чаще на 3% и 2 % соответственно<br>